In [1]:
from Helpers import edgar_support
import re

# Set Up

In [2]:
edgar_filing_url = "https://www.sec.gov/Archives/edgar/data/1093691/000155837021010611/0001558370-21-010611-index.htm"
base_url, filing_summary_url = edgar_support.convert_filing_to_folder(edgar_filing_url=edgar_filing_url)
slugs = edgar_support.retrieve_face_report_slugs(filing_summary_url)

# Balance Sheet Analysis

In [3]:
balance_sheet_url = base_url + "/" + slugs["BS"]
bs = edgar_support.get_clean_table(balance_sheet_url)
bs = bs.pipe(edgar_support.remove_rows_of_zeros)

divisor_caption = [caption for caption in bs["Captions"] if re.search(r"total", caption.lower()) and re.search(r"equity", caption.lower())][0]
bs = edgar_support.common_size_financial_statement(bs, divisor=bs.query("Captions == @divisor_caption").select_dtypes("number").iloc[0])

Unable to clean column - Captions.


# Income Statement Analysis

In [4]:
income_statement_url = base_url + "/" + slugs["IS"]
is_df = edgar_support.get_clean_table(income_statement_url)

is_df = is_df.query("~Captions.fillna('').str.contains('\[')") 
is_df = is_df.query("Captions.notnull()")

is_df = is_df.dropna(thresh=len(is_df)*.1, axis=1)
is_df = is_df.pipe(edgar_support.clean_dataframe)
is_df = is_df.pipe(edgar_support.remove_rows_of_zeros)

is_df = is_df.drop_duplicates(subset=["Captions"], keep="first")

is_df = edgar_support.common_size_financial_statement(is_df, divisor=is_df.select_dtypes("number").iloc[0])

Unable to clean column - Captions.
Unable to clean column - Captions.


# Cash Flow Analysis

In [16]:
cash_flow_url = base_url + "/" + slugs["CF"]
cf = edgar_support.get_clean_table(cash_flow_url)
cf = cf.pipe(edgar_support.remove_rows_of_zeros)
cf = edgar_support.common_size_financial_statement(cf, divisor=cf.select_dtypes("number").iloc[0])

Unable to clean column - Captions.


# Visualizations

## Balance Sheet

In [6]:
bs.style\
    .background_gradient(cmap='Blues', axis=0,)\
    .set_properties(**{'padding': '10px'})\
    .format({col: '{:,.2%}'.format for col in bs.select_dtypes("number").columns })\


,Captions,"Jun. 30, 2021","Dec. 31, 2020"
1,Cash and cash equivalents,65.42%,89.47%
2,Restricted cash,1.69%,4.37%
3,"Available-for-sale securities, at fair value (amortized cost $1,244,618 and allowance for credit losses of $0 at June 30, 2021)",25.72%,0.00%
4,Equity securities,2.49%,0.00%
5,Accounts receivable,1.89%,2.93%
6,Inventory,4.34%,9.50%
7,Prepaid expenses and other current assets,1.25%,3.02%
8,Total current assets,102.81%,109.29%
9,Restricted cash,7.20%,17.58%
10,"Property, plant, and equipment, net",2.29%,5.08%


## Income Statement

In [15]:
is_df.style\
    .background_gradient(cmap='Blues', axis=0,)\
    .set_properties(**{'padding': '10px'})\
    .hide_index()\
    .format({col: '{:,.2%}'.format for col in is_df.select_dtypes("number").columns })\


Captions,"3 Months Ended Jun. 30, 2021","3 Months Ended Jun. 30, 2020","6 Months Ended Jun. 30, 2021","6 Months Ended Jun. 30, 2020"
Net revenue,100.00%,100.00%,100.00%,100.00%
Total cost of revenue,132.35%,100.02%,126.70%,108.93%
Gross loss,-32.35%,-0.02%,-26.70%,-8.93%
Research and development,9.03%,7.17%,10.68%,8.87%
"Selling, general and administrative",31.03%,31.83%,32.69%,30.10%
Change in fair value of contingent consideration,-0.45%,0.00%,0.12%,0.00%
Total operating expenses,39.61%,39.00%,43.48%,38.97%
Operating loss,-71.96%,-39.02%,-70.18%,-47.90%
Interest,-8.24%,-19.66%,-11.47%,-23.12%
"Other expense, net",-0.06%,-0.14%,-0.14%,-0.14%


## Cash Flow

In [17]:
cf.style\
    .background_gradient(cmap='RdBu', axis=0,)\
    .set_properties(**{'padding': '10px'})\
    .format({col: '{:,.2%}'.format for col in cf.select_dtypes("number").columns })

,Captions,"3 Months Ended Jun. 30, 2021","6 Months Ended Jun. 30, 2021","6 Months Ended Jun. 30, 2020"
1,Net loss attributable to the Company,-100.00%,-100.00%,-100.00%
3,Depreciation of long-lived assets,0.00%,6.06%,12.96%
4,Amortization of intangible assets,0.40%,0.46%,0.85%
5,Stock-based compensation,0.00%,12.98%,13.21%
6,Gain on extinguishment of debt,0.00%,0.00%,-28.23%
7,Amortization of debt issuance costs and discount on convertible senior notes,0.00%,1.08%,13.94%
8,Provision for common stock warrants,0.00%,2.15%,17.05%
9,Income tax benefit,0.00%,0.00%,-37.09%
10,Loss on service contracts,0.00%,2.74%,0.59%
11,Fair value adjustment to contingent consideration,0.00%,-0.14%,0.00%
